<!-- Initialize **API** **managers** -->

In [1]:
import os
from Powerfleet_APIs_Management import PowerFleetAPIsManager as ApiManager
from colorama import Fore, Style
import sys  # For printing caught exceptions
import re
import json
from datetime import datetime


print(Fore.YELLOW + f"MAIN CWD={os.getcwd()}")
print(f"MAIN, LISTDIR={os.listdir()}" + Style.RESET_ALL)

# Define the ApiParameterExtractor class
class ApiParameterExtractor:
    def __init__(self, json_file):
        with open(json_file, 'r') as file:
            self.parameters = json.load(file)

    def extract_parameters(self, api_type):
        # Normalize API type to lowercase
        api_type = api_type.lower()
        
        # Validate API type
        if api_type not in self.parameters:
            raise ValueError(f"Invalid API type: {api_type}. Valid types are 'live_api' or 'snapshot_api'.")
        
        # Return the corresponding dictionary for the given API type
        return self.parameters[api_type]


extractor = ApiParameterExtractor("PARAMETERS.json")

# Extract parameters for live_api and snapshot_api
live_api_params         = extractor.extract_parameters("live_api")
snapshot_api_params     = extractor.extract_parameters("snapshot_api")

# Pass the extracted parameters to PowerFleetAPIsManager
live_api_manager        = ApiManager(live_api_params)
snapshot_api_manager    = ApiManager(snapshot_api_params)




MAIN CWD=/home/georger/WorkDocuments/Sxoli/Ptyxiaki/Ptyxiaki-Repository/Python_Scripts/Data_Analysis
MAIN, LISTDIR=['Powerfleet_APIs_Management.py', 'PARAMETERS.json', '__pycache__', 'data_analysis.ipynb']


(Optional) Update mongoDB for visualizing coordinates

In [ ]:
from Powerfleet_APIs_Management import MongoDBConnector as DBConnector


connector = DBConnector()
if connector.check_connection():
    print("Proceed with operations.")
else:
    print("Fix connection issues before proceeding.")

Connection to MongoDB is successful!
Proceed with operations.


Convert **JSON** to **CSV** so that **pandas** be able to use it. Also, delete the <u>JSON</u> files


In [ ]:
import json
import pandas as pd
import os

# Get the Live API data
live_data = live_api_manager.get_live_data()

# Prepare and get Snapshot API data
vehicleId = snapshot_api_params["vehicleId"]
startDate = snapshot_api_params["startDate"]
endDate = snapshot_api_params["endDate"]
snapshot_data = snapshot_api_manager.get_snapshot_data(vehicleId, startDate, endDate)

# Parse JSON strings into dictionaries or lists
try:
    # Attempt to decode Live API data
    live_data_dict = json.loads(live_data) if isinstance(live_data, str) else live_data
except json.JSONDecodeError as e:
    print(f"Error decoding Live JSON: {e}")
    live_data_dict = {}

try:
    # Attempt to decode Snapshot API data
    snapshot_data_dict = json.loads(snapshot_data) if isinstance(snapshot_data, str) else snapshot_data
except json.JSONDecodeError as e:
    print(f"Error decoding Snapshot JSON: {e}")
    snapshot_data_dict = {}

# Handle Live Data: Convert to a list of dictionaries
if isinstance(live_data_dict, dict):
    live_data_list = [value for key, value in live_data_dict.items()]
elif isinstance(live_data_dict, list):
    live_data_list = live_data_dict  # Already a list of dictionaries
else:
    live_data_list = []  # Fallback to empty list if structure is unexpected

# Handle Snapshot Data: Convert to a list of dictionaries
if isinstance(snapshot_data_dict, dict):
    snapshot_data_list = [value for key, value in snapshot_data_dict.items()]
elif isinstance(snapshot_data_dict, list):
    snapshot_data_list = snapshot_data_dict  # Already a list of dictionaries
else:
    snapshot_data_list = []  # Fallback to empty list if structure is unexpected

# Create DataFrames from the lists
live_df = pd.DataFrame(live_data_list)
snapshot_df = pd.DataFrame(snapshot_data_list)

# Save DataFrames to CSV files
output_dir = "../../DataSets"

# Ensure the directory exists
os.makedirs(output_dir, exist_ok=True)

live_data_path = os.path.join(output_dir, "Live_API_DataSets.csv")
snapshot_data_path = os.path.join(output_dir, "Snapshot_API_Response_data_set.csv")

live_df.to_csv(live_data_path, index=False)
snapshot_df.to_csv(snapshot_data_path, index=False)

print(f"Live data saved to {live_data_path}")
print(f"Snapshot data saved to {snapshot_data_path}")


try:
    # Delete the file
    file_path.unlink()
    print(f"{file_path} has been deleted successfully.")
except FileNotFoundError:
    print(f"{file_path} does not exist.")
except PermissionError:
    print(f"Permission denied: Unable to delete {file_path}.")
except Exception as e:
    print(f"An error occurred: {e}")


It's Live API
API Request Successful!
It's Snapshot API
Request Body: {
    "vehicleId": "7",
    "startDate": "2024-01-01 00:00:00",
    "endDate": "2024-11-23 23:59:59"
}

<Response [200]>
API Request Successful!
Live data saved to ../../DataSets/Live_API_DataSets.csv
Snapshot data saved to ../../DataSets/Snapshot_API_Response_data_set.csv


**Sort** <u>dates and plateId</u>


Visualize data **dispersion**